# Using `fmriprep` on the SLURM cluster



--------------------


#### HISTORY

* 7/9/2021 - dcosme revised for bbprime
* 5/22/20 - mbod updated for revised MURI fmriprep pipeline
* 1/22/20 - matt, jeesung and nicole initial test setup for using SLURM

In [2]:
import os

## 1. Logging onto the SLURM master


* master name: `cls000.asc.upenn.edu`


```
ssh <JANUS_UN>@asc.upenn.edu@cls000
```


* This will give you a terminal on the SLURM master node where you can look at the process queue and schedule jobs.

## 2. Creating job files

* Each process that you send to SLURM needs a job file, which is a text file with the commands to be executed by SLURM

* For the bbprime project these job files should be placed in:

```
/data00/slurm_jobs/slurm_bbprime
```

* A job file has:

    1. some SLURM directives in lines beginning `#SBATCH`
    2. one or more commands to be run (prefixed with `srun`)

In [5]:
cat /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP00.job

#!/bin/bash
#SBATCH --job-name=fmriprep_sub-BPP00.job
#SBATCH --output=out/fmriprep_sub-BPP00.out
#SBATCH --error=out/fmriprep_sub-BPP00.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/bbprime/data/bids_data:/data \
    -B /data00/projects/bbprime/data/bids_data/derivatives:/out  \
    -B /data00/projects/bbprime/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    --use-aroma \
    participant --participant_label sub-BPP00


* To create a new job file you can copy an existing one and replace the participant ID (e.g. `sub-MURIP376`) everywhere it is stated.

* Also check the `/data` mapping in the singularity command

### Define variables

In [6]:
project = 'bbprime'
project_dir = os.path.join('/data00/projects/' + project)
slurm_dir = os.path.join('/data00/slurm_jobs/slurm_' + project)

### Make the directory if it doesn't exist

In [7]:
!mkdir $slurm_dir

mkdir: cannot create directory ‘/data00/slurm_jobs/slurm_bbprime’: File exists


### Create job template

In [8]:
job_file_template = r'''#!/bin/bash
#SBATCH --job-name=fmriprep_{ID}.job
#SBATCH --output=out/fmriprep_{ID}.out
#SBATCH --error=out/fmriprep_{ID}.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/bbprime/data/bids_data:/data \
    -B /data00/projects/bbprime/data/bids_data/derivatives:/out  \
    -B /data00/projects/bbprime/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    --use-aroma \
    participant --participant_label {ID} \
    --skip-bids-validation
'''


In [9]:
print(job_file_template.format(ID='sub-BPP00'))

#!/bin/bash
#SBATCH --job-name=fmriprep_sub-BPP00.job
#SBATCH --output=out/fmriprep_sub-BPP00.out
#SBATCH --error=out/fmriprep_sub-BPP00.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/bbprime/data/bids_data:/data \
    -B /data00/projects/bbprime/data/bids_data/derivatives:/out  \
    -B /data00/projects/bbprime/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    --use-aroma \
    participant --participant_label sub-BPP00 \
    --skip-bids-validation



In [10]:
subjs_to_process = [s for s in os.listdir(os.path.join(project_dir, 'data/bids_data'))
                       if s.startswith('sub-BP')]

existing_job_files = [s for s in os.listdir(slurm_dir)
                       if s.endswith('.job') and s.count('sub-BP')==1]

In [47]:
existing_job_files, subjs_to_process

(['fmriprep_sub-BPP00.job',
  'fmriprep_sub-BPP01.job',
  'fmriprep_sub-BPP05.job',
  'fmriprep_sub-BPP06.job'],
 ['sub-BPP00', 'sub-BPP01', 'sub-BPP05', 'sub-BPP06'])

In [11]:
for s in subjs_to_process:
    file_path = os.path.join(slurm_dir, 'fmriprep_{}.job').format(s)
    print('Creating', file_path)

    with open(file_path.format(s), 'w') as job:
        job.write(job_file_template.format(ID=s)
                     )


Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP00.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP01.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP05.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP06.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP07.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP11.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP13.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP19.job
Creating /data00/slurm_jobs/slurm_bbprime/fmriprep_sub-BPP21.job


## Scheduling a job to SLURM

* Use the `sbatch` command

    ```
    sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 name_of_job_file
    
    ```

    * `-D` is the working directory
    * `-c` the number of processors to use
    * then the name of the job file
    

### Create log file directory

If this doesn't exist, your job will fail without explanation

In [49]:
!mkdir $slurm_dir/out

mkdir: cannot create directory ‘/data00/slurm_jobs/slurm_bbprime/out’: File exists


### Run all jobs
To submit all the jobs, copy and past the following lines of code into the terminal after logging into the slurm cluster

In [50]:
print("Alternatively, submit the whole batch by copying and pasting the following lines in terminal:")
print("")

print(f"cd {slurm_dir}")
for s in subjs_to_process:
    print(f"sbatch -D {slurm_dir} -c 8 fmriprep_{s}.job")
print(" ")

Alternatively, submit the whole batch by copying and pasting the following lines in terminal:

cd /data00/slurm_jobs/slurm_bbprime
sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 fmriprep_sub-BPP00.job
sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 fmriprep_sub-BPP01.job
sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 fmriprep_sub-BPP05.job
sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 fmriprep_sub-BPP06.job
 
